# This file implements overlap function

In [1]:
import numpy as np
import math

# For testing purposes
sensors = [{'id': 1, 'type': 1, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 200}, 'price': 100}, {'id': 2, 'type': 2, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 500}, 'price': 200}, {'id': 3, 'type': 3, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 700}, 'price': 100}, {'id': 4, 'type': 1, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 1000}, 'price': 10}, {'id': 5, 'type': 2, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 1000}, 'price': 100}]

In [5]:
# Case 1
# We obtain the maximum overlap in the case that the middle line of the fields of views are on the same line and on the same car surface

def sensor_inside_sensor(car_p1, env_p1, car_p2, env_p2):
    if car_p2.all() == env_p2.all():
        return True
    if car_p1.all() == car_p2.all():
        line = (car_p1 - env_p1)/(car_p2 - env_p2)
        for i in range(len(line) - 1):
            if line[i] != line[i+1]:
                return False
        else:
            return True
    return False

In [6]:
def overlap(b1, b2):
    car_p1 = np.array([int(x) for x in b1[0]])
    env_p1 = np.array([int(x) for x in b1[1]])
    id1 = int(b1[2])
    
    car_p2 = np.array([int(x) for x in b2[0]])
    env_p2 = np.array([int(x) for x in b2[1]])
    id2 = int(b2[2])
    
    if sensor_inside_sensor(car_p1, env_p1, car_p2, env_p2):
        return 1
    
    cone_axis1 = env_p1 - car_p1
    cone_axis2 = env_p2 - car_p2
    
    # Angle between the axis
    
    u_cone_axis1 = cone_axis1 / np.linalg.norm(cone_axis1)
    u_cone_axis2 = cone_axis2 / np.linalg.norm(cone_axis2)
    
    axis_angle = np. arccos(np.dot(u_cone_axis1, u_cone_axis2))
    
    #print(math.degrees(axis_angle))
    
    # Distance between the middle points of the vectors cone_axis1 and cone_axis2
    
    mid_cone_axis1 = (np.linalg.norm(cone_axis1)/2)*u_cone_axis1
    mid_cone_axis2 = (np.linalg.norm(cone_axis2)/2)*u_cone_axis2
    
    mid_point_dist = np.linalg.norm(mid_cone_axis1 - mid_cone_axis2)
    
    top_point_dist = np.linalg.norm(env_p1 - env_p2)
    
    #print(mid_point_dist)
    
    sensor1, sensor2 = None, None
    
    for sensor in sensors:
        
        if sensor['id'] == id1:
            sensor1 = sensor
        
        if sensor['id'] == id2:
            sensor2 = sensor
            
    # This part of the code is very heuristical and possibly works badly and ruins everything
    
    side1 = np.linalg.norm(cone_axis1)
    side2 = np.linalg.norm(cone_axis2)
    
    top_half1_dist1 = math.tan(math.radians(sensor1["view"]["angle"]["horizontal"])/2)*side1
    top_half1_dist2 = math.tan(math.radians(sensor1["view"]["angle"]["vertical"])/2)*side1
    
    top_half2_dist1 = math.tan(math.radians(sensor2["view"]["angle"]["horizontal"])/2)*side2
    top_half2_dist2 = math.tan(math.radians(sensor2["view"]["angle"]["vertical"])/2)*side2
    
    #print("Distances on top: ", top_half1_dist1, top_half1_dist2, top_half2_dist1, top_half2_dist2)
    
    mid_half1_dist1 = top_half1_dist1/2
    mid_half1_dist2 = top_half1_dist2/2
    
    mid_half2_dist1 = top_half2_dist1/2
    mid_half2_dist2 = top_half2_dist2/2
    
    #print("Distances in middle: ", mid_half1_dist1, mid_half1_dist2, mid_half2_dist1, mid_half2_dist2)
    
    difference_top1 = top_half1_dist1 + top_half2_dist1 - top_point_dist
    difference_top2 = top_half1_dist2 + top_half2_dist2 - top_point_dist
    
    difference_mid1 = mid_half1_dist1 + mid_half2_dist1 - mid_point_dist
    difference_mid2 = mid_half1_dist2 + mid_half2_dist2 - mid_point_dist
    
    #print("Top differences: ", difference_top1, difference_top2)
    #print("Middle differences: ", difference_mid1, difference_mid2)
    
    top_divisor1 = max([top_half1_dist1, top_half2_dist1])
    top_divisor2 = max([top_half1_dist2, top_half2_dist2])
    
    top_divisor = top_divisor1 + top_divisor2
    
    mid_divisor1 = max([mid_half1_dist1, mid_half2_dist1])
    mid_divisor2 = max([mid_half1_dist2, mid_half2_dist2])
    
    mid_divisor = mid_divisor1 + mid_divisor2
    
    top_sum = 0
    mid_sum = 0
    
    if difference_top1 > 0:
        top_sum += difference_top1
    if difference_top2 > 0:
        top_sum += difference_top2
    if difference_mid1 > 0:
        mid_sum += difference_mid1
    if difference_mid2 > 0:
        mid_sum += difference_mid2
    
    overlap_result = (top_sum + mid_sum)/(top_divisor + mid_divisor)
    
    print("Final result: ", overlap_result)
    
    if overlap_result > 1:
        return 1/overlap_result
    
    return overlap_result

In [7]:
overlap(((-2200, -550, 1745), (-2308, -603, 1608), 1), ((-2200, -550, 1745), (-2286, -648, 1592), 1))

1